# Query ESGF input4MIPs index and extract source_id entries

## notes

## imports

In [1]:
%%time
import json
import numpy as np
import os
import pdb
import requests
from IPython.display import clear_output

CPU times: user 64.8 ms, sys: 36.2 ms, total: 101 ms
Wall time: 161 ms


## example source_id queries

## get source_id entries

In [2]:
%%time
# both input4MIPs and input4mips
project = "input4MIPs"
actId = "input4MIPs"

# Dataset search
inputsD = "https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&" \
         "fq=type:Dataset&fq=replica:false&fq=activity_id:" + project + "&facet.field=source_id"
js = requests.get(inputsD)
js_mipsD = json.loads(js.text)
print(inputsD)

# File search
inputsF = "https://esgf-node.llnl.gov/solr/files/select?q=*:*&rows=0&wt=json&facet=true&" \
          "fq=type:File&fq=replica:false&fq=activity_id:" + project + "&facet.field=source_id"
js = requests.get(inputsF)
js_mipsF = json.loads(js.text)
print(inputsF)

https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&fq=type:Dataset&fq=replica:false&fq=activity_id:input4MIPs&facet.field=source_id
https://esgf-node.llnl.gov/solr/files/select?q=*:*&rows=0&wt=json&facet=true&fq=type:File&fq=replica:false&fq=activity_id:input4MIPs&facet.field=source_id
CPU times: user 37.2 ms, sys: 9.76 ms, total: 46.9 ms
Wall time: 298 ms


## extract source_id entries and composite

In [3]:
dicInpmD = js_mipsD["facet_counts"]["facet_fields"]["source_id"]
print("len(dicInpmD):", len(dicInpmD))
dicInpmF = js_mipsF["facet_counts"]["facet_fields"]["source_id"]
print("len(dicInpmF):", len(dicInpmF))
#print(dicInpmD)
#print(dicInpmF)

len(dicInpmD): 370
len(dicInpmF): 348


In [4]:
# datasets
srcIdLen = len(dicInpmD)
print("srcIdDLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
srcIdDDict = {}
srcIds = dicInpmD  # reset to generic variable name
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdDDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdDDict.keys()):", len(srcIdDDict.keys()))
print("dataset counts:", counts)
# sort dictionary
#sorted(srcIdDDict.items())
#print(srcIdDDict)
srcIdDDictList = list(sorted(srcIdDDict.keys()))
#print(srcIdDDictList)

# files
srcIdLen = len(dicInpmF)
print("srcIdFLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
srcIdFDict = {}
srcIds = dicInpmF  # reset to generic variable name
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdFDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdFDict.keys()):", len(srcIdFDict.keys()))
print("dataset counts:", counts)
# sort dictionary
srcIdFDictList = list(sorted(srcIdFDict.keys()))
#print(srcIdFDictList)

# determine missing
print("Search results: Dataset includes, excluded from File searches (likely latest:false):")
set(srcIdDDictList).difference(srcIdFDictList)

srcIdDLen: 370
len(srcIdDDict.keys()): 185
dataset counts: 5897
srcIdFLen: 348
len(srcIdFDict.keys()): 174
dataset counts: 10384
Search results: Dataset includes, excluded from File searches:


{'CCMI-hist-nat-1-0',
 'CCMI-hist-sol-1-0',
 'CCMI-hist-volc-1-0',
 'IAMC-AIM-ssp370-1-0',
 'IAMC-GCAM4-ssp434-1-0',
 'IAMC-GCAM4-ssp460-1-0',
 'IAMC-IMAGE-ssp119-1-0',
 'IAMC-IMAGE-ssp126-1-0',
 'IAMC-MESSAGE-GLOBIOM-ssp245-1-0',
 'IAMC-REMIND-MAGPIE-ssp534-over-1-0',
 'IAMC-REMIND-MAGPIE-ssp585-1-0'}

## example source_id queries

## using source_id entries from Dataset search build a library

In [5]:
solrQry = "https://esgf-node.llnl.gov/esg-search/search/?limit=1000&format=application%2Fsolr%2Bjson&source_id=" \
          "PLACEHOLDER" + "&project=input4mips&project=input4MIPs&distrib=false&fields=*"  # all fields
mstrJson = {}  # create catch dictionary
oF = "tmp.json"
for count, srcId in enumerate(srcIdDDict.keys()):
    print(count, srcId)
    qryStr = solrQry.replace("PLACEHOLDER", srcId)
    #print("qryStr:", qryStr)
    js = requests.get(qryStr)
    js_srcId = json.loads(js.text)
    #print(js_srcId["response"]["docs"][0])
    # write to placeholder to test
    srcIdLen = len(js_srcId["response"]["docs"])
    clear_output(wait=False)
    # https://stackoverflow.com/questions/24816237/ipython-notebook-clear-cell-output-in-code
    print("len(js_srcId):", srcIdLen)
    for entry in np.arange(0, srcIdLen):
        a = js_srcId["response"]["docs"][entry]
        instId = a["instance_id"]
        mstrJson[instId] = a        
        if os.path.exists(oF):
            os.remove(oF)
        fH = open(oF, "w")
        json.dump(a, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
        #json.dump(mstrJson, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
        fH.close()
    #pdb.set_trace()
print("All done")
# cleanup
os.remove(oF)  # cleanup
# Write all out
oF = "comp.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(mstrJson, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

len(js_srcId): 247
All done


## create input4MIPs_source_id.json

In [6]:
srcIdDict = {}
srcIdDict["source_id"] = {}
keyList = [
    "_timestamp", "contact", "dataset_category", "data_node", "datetime_start",
    "datetime_stop", "latest", "frequency", "institution_id", "mip_era", "realm",
    "replica", "source_id", "source_version", "target_mip", "version", "xlink"
       ]
# latest (bool)/deprecated (string)

# loop through entries
for c1, a in enumerate(mstrJson):
    tmp = mstrJson[a]
    srcId = tmp["source_id"][0]
    print(srcId)
    if srcId not in srcIdDict.keys():
        dic = [tmp.get(key) for key in keyList]
        srcIdDict["source_id"][srcId] = {}
        for c2, b in enumerate(keyList):
            c = dic[c2]
            if isinstance(c, list):
                c = c[0]
            srcIdDict["source_id"][srcId][keyList[c2]] = c
      
clear_output(wait=False)
print("len(srcIdDict[\"source_id\"].keys()):", len(srcIdDict["source_id"].keys()))
# Write all out
oF = "../input4MIPs_source_id.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(srcIdDict, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

len(srcIdDict["source_id"].keys()): 185


In [8]:
srcIdDict.keys()

dict_keys(['source_id'])

## augment source_id with values from input4mips-cmor-tables

In [24]:
%%time
# read input4mips-cmor-tables holdings
srcIdGithub = "https://raw.githubusercontent.com/PCMDI/input4MIPs-cmor-tables/master/input4MIPs_source_id.json"
js = requests.get(srcIdGithub)
srcId = json.loads(js.text)

# get required_global_attributes and prepopulate - controlled values
# iterate and add entries in input4mip-cmor-tables/input4MIPs_source_id.json - controlled and uncontrolled values

reqGlobAtt = ["activity_id", "contact", "dataset_category", "frequency",
              "further_info_url", "grid_label", "institution_id", "license", "mip_era",
              "nominal_resolution", "realm", "region", "source", "source_id", "source_version", 
              "target_mip", "title", "variable_id"]
# file "Conventions", "creation_date", "tracking_id", 
# irrelevant "institution", "table_id",
# esgf fields
esgfAtt = ["_timestamp", "data_node", "latest", "replica", "version", "xlink"]

CPU times: user 7.24 ms, sys: 4.16 ms, total: 11.4 ms
Wall time: 320 ms


In [26]:
js_srcId["source_id"].keys()

dict_keys(['ACCESS1-3-rcp85-1-0', 'CCSM4-rcp26-1-0', 'CCSM4-rcp85-1-0', 'CESM2-ssp585-1-0', 'CNRM-CM6-1-ssp126-1-0', 'CNRM-CM6-1-ssp585-1-0', 'CNRM-ESM2-1-ssp585-1-0', 'CSIRO-MK3-6-0-rcp85-1-0', 'HadGEM2-ES-rcp85-1-0', 'IPSL-CM5A-MR-rcp26-1-0', 'IPSL-CM5A-MR-rcp85-1-0', 'MIROC-ESM-CHEM-rcp26-1-0', 'MIROC-ESM-CHEM-rcp85-1-0', 'MIROC5-rcp26-1-0', 'MIROC5-rcp85-1-0', 'MRI-JRA55-do-1-3', 'MRI-JRA55-do-1-3-2', 'MRI-JRA55-do-1-4-0', 'MRI-JRA55-do-1-5-0', 'MRI-JRA55-do-1-6-0', 'NorESM1-M-rcp26-1-0', 'NorESM1-M-rcp85-1-0', 'PCMDI-AMIP-1-1-3', 'PCMDI-AMIP-1-1-4', 'PCMDI-AMIP-1-1-5', 'PCMDI-AMIP-1-1-6', 'PCMDI-AMIP-1-1-7', 'PCMDI-AMIP-1-1-8', 'PCMDI-AMIP-1-1-9', 'UKESM1-0-LL-ssp585-1-0'])

In [7]:
srcIdDict.keys()

dict_keys(['source_id'])

In [45]:
js_srcId["source_id"].keys()

dict_keys(['ACCESS1-3-rcp85-1-0', 'CCSM4-rcp26-1-0', 'CCSM4-rcp85-1-0', 'CESM2-ssp585-1-0', 'CNRM-CM6-1-ssp126-1-0', 'CNRM-CM6-1-ssp585-1-0', 'CNRM-ESM2-1-ssp585-1-0', 'CSIRO-MK3-6-0-rcp85-1-0', 'HadGEM2-ES-rcp85-1-0', 'IPSL-CM5A-MR-rcp26-1-0', 'IPSL-CM5A-MR-rcp85-1-0', 'MIROC-ESM-CHEM-rcp26-1-0', 'MIROC-ESM-CHEM-rcp85-1-0', 'MIROC5-rcp26-1-0', 'MIROC5-rcp85-1-0', 'MRI-JRA55-do-1-3', 'MRI-JRA55-do-1-3-2', 'MRI-JRA55-do-1-4-0', 'MRI-JRA55-do-1-5-0', 'MRI-JRA55-do-1-6-0', 'NorESM1-M-rcp26-1-0', 'NorESM1-M-rcp85-1-0', 'PCMDI-AMIP-1-1-3', 'PCMDI-AMIP-1-1-4', 'PCMDI-AMIP-1-1-5', 'PCMDI-AMIP-1-1-6', 'PCMDI-AMIP-1-1-7', 'PCMDI-AMIP-1-1-8', 'PCMDI-AMIP-1-1-9', 'UKESM1-0-LL-ssp585-1-0'])

# Create other CV entries

## create other *.json entries - lift direct from repo

In [21]:
%%time
# read input4mips_CVs
rawPath = "https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/"
CVList = ["DRS", "activity_id", "dataset_category", "license", "mip_era",
          "product", "required_global_attributes", "target_mip", "tracking_id"]
# loop through entries
for count, cv in enumerate(CVList):
    path = "".join([rawPath, "input4MIPs_", cv, ".json"])
    print(path)
    js = requests.get(path)
    vars()[cv] = json.loads(js.text)

https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_DRS.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_activity_id.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_dataset_category.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_institution_id.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_license.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_mip_era.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_product.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_required_global_attributes.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_target_mip.json
https://raw.githubusercontent.com/PCMDI/input4MIPs_CVs/main/input4MIPs_tracking_id.json
CPU times: user 77.2 ms, sys: 21.2 ms, total: 98.4 ms
Wall time: 2.07 s


## create institution_id from source_id entries

In [14]:
# load data
srcIdPath = "../input4MIPs_source_id.json"
with open(srcIdPath, 'r') as fH:
    js_srcId = json.load(fH)

#js_srcId["source_id"].keys()
#js_srcId["source_id"]["ACCESS1-3-rcp85-1-0"]["institution_id"]

In [25]:
# iterate through and catch
instId = []
for count, key in enumerate(js_srcId["source_id"].keys()):
    instIdTmp = js_srcId["source_id"][key]["institution_id"]
    #print(count, key, instIdTmp)
    if instIdTmp not in instId:
        instId.append(instIdTmp)
instId.sort()
instId

['CCCma',
 'CNRM-Cerfacs',
 'IACETH',
 'IAMC',
 'ImperialCollege',
 'MOHC',
 'MPI-B',
 'MPI-M',
 'MRI',
 'NASA-GSFC',
 'NCAR',
 'NCAS',
 'PCMDI',
 'PNNL-JGCRI',
 'SOLARIS-HEPPA',
 'UCI',
 'UColorado',
 'UReading',
 'UoM',
 'UofMD',
 'VUA']

In [26]:
# write to input4MIPs_institution_id.json
tmp = {}
tmp["institution_id"] = instId
# Write all out
oF = "../input4MIPs_institution_id.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(tmp, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

## create composite CV

In [41]:
CVExportList = ["activity_id", "dataset_category", "DRS", "institution_id", "license", "mip_era",
                "product", "required_global_attributes", "source_id", "target_mip", "tracking_id"]
CVExportList.sort()

# Collate all CVs in master
tmp = {}
tmp["CV"] = {}
for key in CVExportList:
    fileName = "".join(["input4MIPs_", key, ".json"])
    filePath = os.path.join("..", fileName)
    #print(filePath)
    with open(filePath,'r') as fH:
        keyDict = json.load(fH)
        tmp["CV"][key] = keyDict[key]

# Write collated CVs to file
if not os.path.exists("../CVs"):
    os.makedirs("../CVs")
oF = "../CVs/input4MIPs_CV.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(tmp, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

In [34]:
tmp["CV"].keys()

dict_keys(['DRS', 'activity_id', 'dataset_category', 'institution_id', 'license', 'mip_era', 'product', 'required_global_attributes', 'source_id', 'target_mip', 'tracking_id'])